In [ ]:
!pip install streamlit pdfplumber python-docx sentence-transformers faiss-cpu transformers pyngrok


In [ ]:
# Install necessary packages. This may take a few minutes.
!pip install -q streamlit pdfplumber sentence-transformers faiss-cpu transformers torch numpy pandas scikit-learn tqdm pyngrok

# Optional OCR support (only if you want to handle scanned PDFs)
!apt-get update -qq
!apt-get install -y -qq poppler-utils tesseract-ocr
!pip install -q pytesseract pillow


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("3266z6lGp0aq7ZuFnK3IeT09QAG_6ueizARhmhDuvN7aXJ8yJ")

In [ ]:
# Optional: set your ngrok auth token if you have one for more stable tunnels.
from pyngrok import ngrok
# ngrok.set_auth_token("3266z6lGp0aq7ZuFnK3IeT09QAG_6ueizARhmhDuvN7aXJ8yJ")   # <-- (optional) uncomment & put your token

# Kill previous tunnels (safe step) and create a new tunnel to port 8501
ngrok.kill()
public_url = ngrok.connect(addr='8501')
print("Ngrok public URL (open this after streamlit starts):", public_url)


In [ ]:
%%writefile app.py
import streamlit as st
import pdfplumber
import numpy as np
import faiss
import tempfile
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import torch
from fpdf import FPDF

# ------------------ Page Config ------------------
st.set_page_config(page_title="AI PDF Assistant", page_icon="📄", layout="wide")

# ------------------ Custom CSS ------------------
st.markdown("""
<style>
    .main {
        background-color: #f5f7fa;
    }
    .stButton button {
        background-color: #4CAF50;
        color: white;
        border-radius: 8px;
        padding: 0.5em 1em;
        font-weight: bold;
    }
    .stTextInput input, .stTextArea textarea {
        border: 2px solid #4CAF50;
        border-radius: 8px;
    }
    .card {
        background-color: white;
        padding: 20px;
        border-radius: 12px;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        margin-bottom: 20px;
    }
</style>
""", unsafe_allow_html=True)

# ------------------ Utilities ------------------
@st.cache_data
def extract_text_from_pdf(file_path_or_buffer):
    text = ""
    with pdfplumber.open(file_path_or_buffer) as pdf:
        for page in pdf.pages:
            p = page.extract_text()
            if p:
                text += p + "\n"
    return text

def chunk_text(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    length = len(text)
    while start < length:
        end = min(start + chunk_size, length)
        chunk = text[start:end]
        if chunk.strip():
            chunks.append(chunk.strip())
        if end == length:
            break
        start = max(end - overlap, 0)
    return chunks

@st.cache_resource
def load_models():
    device_for_pipeline = 0 if torch.cuda.is_available() else -1
    embed_device = "cuda" if torch.cuda.is_available() else "cpu"

    embed_model = SentenceTransformer('all-MiniLM-L6-v2', device=embed_device)
    summarizer = pipeline('summarization', model='sshleifer/distilbart-cnn-12-6', device=device_for_pipeline)
    qa_pipeline = pipeline('question-answering', model='deepset/roberta-base-squad2', device=device_for_pipeline)

    return embed_model, summarizer, qa_pipeline

def build_faiss_index(embeddings: np.ndarray):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings.astype('float32'))
    return index

def retrieve_top_k(question, embed_model, index, chunks, k=5):
    q_emb = embed_model.encode([question], convert_to_numpy=True).astype('float32')
    D, I = index.search(q_emb, k)
    retrieved = []
    for idx in I[0]:
        if idx >= 0 and idx < len(chunks):
            retrieved.append(chunks[idx])
    return retrieved

def make_context(retrieved_chunks, max_chars=3000):
    ctx = ""
    for c in retrieved_chunks:
        if len(ctx) + len(c) > max_chars:
            break
        ctx += c + "\n"
    return ctx
def save_summary_as_pdf(summary_text, filename="summary.pdf"):
    from fpdf import FPDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    # Use downloaded font from /content
    pdf.add_font('DejaVu', '', '/content/DejaVuSans.ttf', uni=True)
    pdf.set_font("DejaVu", size=12)
    for line in summary_text.split("\n"):
        pdf.multi_cell(0, 10, line)
    pdf.output(filename)
    return filename






# ------------------ Sidebar ------------------
st.sidebar.title("⚙️ Settings")
st.sidebar.info("Navigate between sections:")
menu = st.sidebar.radio("Choose Section", ["📤 Upload PDF", "📝 Summary", "💬 Q&A", "🔧 Settings"])

dark_mode = st.sidebar.checkbox("🌙 Dark Mode")

if dark_mode:
    st.markdown("""<style> body {background-color: #1e1e1e; color: white;} </style>""", unsafe_allow_html=True)

# ------------------ Main Section ------------------
st.title("📄 AI PDF Assistant")

if "doc_text" not in st.session_state:
    st.session_state["doc_text"] = ""
if "chunks" not in st.session_state:
    st.session_state["chunks"] = []
if "index" not in st.session_state:
    st.session_state["index"] = None
if "summary" not in st.session_state:
    st.session_state["summary"] = ""

# 📤 Upload Section
if menu == "📤 Upload PDF":
    st.header("Upload a PDF Document")
    uploaded_file = st.file_uploader("Choose your PDF", type=['pdf'])

    if uploaded_file is not None:
        tfile = tempfile.NamedTemporaryFile(delete=False, suffix='.pdf')
        tfile.write(uploaded_file.read())
        tfile.flush()

        with st.spinner("Extracting text..."):
            raw_text = extract_text_from_pdf(tfile.name)

        if not raw_text.strip():
            st.error("❌ Could not extract text. (Maybe scanned images)")
        else:
            st.session_state["doc_text"] = raw_text
            st.session_state["chunks"] = chunk_text(raw_text, 1200, 200)

            st.success("✅ Document uploaded and processed!")
            with st.expander("👀 Preview (first 2000 chars)"):
                st.write(raw_text[:2000] + ("..." if len(raw_text) > 2000 else ""))

# 📝 Summary Section
elif menu == "📝 Summary":
    st.header("Generate Summary")
    if not st.session_state["doc_text"]:
        st.warning("⚠️ Please upload a document first.")
    else:
        embed_model, summarizer, qa_pipeline = load_models()

        length = st.radio("Choose Summary Length", ["Short", "Medium", "Long"], horizontal=True)
        if st.button("✨ Generate Summary"):
            with st.spinner("Summarizing..."):
                text = st.session_state["doc_text"]
                chunks = st.session_state["chunks"]

                if len(text) < 4000:
                    max_len = 60 if length=="Short" else (150 if length=="Medium" else 400)
                    summary = summarizer(text, max_length=max_len, min_length=20, truncation=True)[0]['summary_text']
                else:
                    partials = []
                    for c in chunks:
                        p = summarizer(c, max_length=120, min_length=20, truncation=True)[0]['summary_text']
                        partials.append(p)
                    combined = " ".join(partials)
                    max_len = 60 if length=="Short" else (150 if length=="Medium" else 400)
                    summary = summarizer(combined, max_length=max_len, min_length=20, truncation=True)[0]['summary_text']

                st.session_state["summary"] = summary
                st.success("✅ Summary ready!")

        if st.session_state["summary"]:
            st.markdown('<div class="card"><h4>📌 Summary</h4><p>' + st.session_state["summary"] + '</p></div>', unsafe_allow_html=True)
            st.download_button("⬇️ Download as TXT", st.session_state["summary"], file_name="summary.txt")

            pdf_file = save_summary_as_pdf(st.session_state["summary"])
            with open(pdf_file, "rb") as f:
                st.download_button("⬇️ Download as PDF", f, file_name="summary.pdf")

# 💬 Q&A Section
elif menu == "💬 Q&A":
    st.header("Ask Questions about the Document")
    if not st.session_state["chunks"]:
        st.warning("⚠️ Please upload a document first.")
    else:
        embed_model, summarizer, qa_pipeline = load_models()

        # Build FAISS index if not already
        if st.session_state["index"] is None:
            embeddings = embed_model.encode(st.session_state["chunks"], convert_to_numpy=True)
            st.session_state["index"] = build_faiss_index(embeddings)

        question = st.text_area("💬 Type your question here:")
        k = st.slider("Number of chunks to search", 1, 10, 4)

        if st.button("🔍 Get Answer"):
            if not question.strip():
                st.warning("Please enter a question!")
            else:
                with st.spinner("Finding answer..."):
                    retrieved = retrieve_top_k(question, embed_model, st.session_state["index"], st.session_state["chunks"], k=k)
                    context = make_context(retrieved)
                    result = qa_pipeline(question=question, context=context)
                    answer = result.get("answer", "❌ No answer found")

                    st.markdown(f'<div class="card"><h4>✅ Answer</h4><p>{answer}</p></div>', unsafe_allow_html=True)

                    with st.expander("📖 Source Chunks"):
                        for i, r in enumerate(retrieved):
                            st.markdown(f"**Chunk {i+1}:** {r[:400]}{'...' if len(r)>400 else ''}")

# 🔧 Settings Section
elif menu == "🔧 Settings":
    st.header("Settings & Info")
    st.write("🔹 Developed with Streamlit + FAISS + Transformers")
    st.write("🔹 Supports Summary + Q&A from PDFs")
    st.write("🔹 Try Dark Mode from sidebar 🌙")


In [ ]:
!pip install reportlab
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm

def save_summary_as_pdf(summary_text, filename="summary.pdf"):
    c = canvas.Canvas(filename, pagesize=A4)
    width, height = A4
    text_object = c.beginText(2*cm, height - 2*cm)
    text_object.setFont("Helvetica", 12)
    for line in summary_text.split("\n"):
        text_object.textLine(line)
    c.drawText(text_object)
    c.showPage()
    c.save()
    return filename


In [ ]:
!pip install fpdf


In [ ]:
!wget -q https://github.com/dejavu-fonts/dejavu-fonts/raw/master/ttf/DejaVuSans.ttf -O /content/DejaVuSans.ttf


In [ ]:
# Start Streamlit app
get_ipython().system_raw("streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &")

print("🚀 Now open this URL in your browser:", public_url)
https://c9da762445bc.ngrok-free.app/

In [ ]:
import os

# 1️⃣ Project Folder তৈরি
project_path = "/content/ai-pdf-assistant"
os.makedirs(project_path, exist_ok=True)
print("Project folder created at:", project_path)

# 2️⃣ app.py তৈরি (modernized, minimal placeholder)
app_code = """
import streamlit as st

st.title("📄 AI PDF Assistant")
st.write("This is a placeholder for the modern AI PDF Assistant app.")
"""

with open(os.path.join(project_path, "app.py"), "w") as f:
    f.write(app_code)

# 3️⃣ requirements.txt তৈরি
requirements = """
streamlit
pdfplumber
sentence-transformers
faiss-cpu
transformers
torch
numpy
pandas
scikit-learn
tqdm
pyngrok
reportlab
"""
with open(os.path.join(project_path, "requirements.txt"), "w") as f:
    f.write(requirements.strip())

# 4️⃣ README.md তৈরি
readme = """
# AI PDF Assistant

A modern Streamlit app to:

- Upload PDF
- Generate summaries
- Ask questions (Q&A)
- Download summary as TXT or PDF
- Modern UI with sidebar and dark mode

## Run locally / Colab

pip install -r requirements.txt
streamlit run app.py
"""
with open(os.path.join(project_path, "README.md"), "w") as f:
    f.write(readme.strip())

# 5️⃣ .gitignore তৈরি
gitignore = """
__pycache__/
*.pyc
*.pkl
*.ipynb_checkpoints/
.env
"""
with open(os.path.join(project_path, ".gitignore"), "w") as f:
    f.write(gitignore.strip())

# ✅ Check files
print("Files created:")
!ls -l /content/ai-pdf-assistant


In [ ]:
# 1️⃣ Navigate to project folder
!cd /content/ai-pdf-assistant

# 2️⃣ Initialize Git repository (if not done already)
!git init

# 3️⃣ Add all files to staging
!git add .

# 4️⃣ Commit changes
!git commit -m "Initial commit: Modern AI PDF Assistant"

# 5️⃣ Add GitHub remote
!git remote add origin https://github.com/kazisaymon/Pdf-assistant.git
# যদি আগে remote থাকে, replace করতে:
# git remote set-url origin https://github.com/kazisaymon/Pdf-assistant.git

# 6️⃣ Rename branch to main
!git branch -M main

# 7️⃣ Push to GitHub
# প্রথমবার push করলে GitHub username এবং Personal Access Token ব্যবহার করুন
!git push -u origin main

To fix the `Author identity unknown` error, run the following commands to set your global Git user name and email. Replace `"Your Name"` and `"you@example.com"` with your actual name and email.

In [ ]:
!git config --global user.name "saymon800"
!git config --global user.email "kaaziisaymon58@gmail.com"

After setting your Git identity, you can try running the push command (`!git push -u origin main`) again.

If you encounter the `error: failed to push some refs` again, it's likely an authentication issue with GitHub. You may need to use a [Personal Access Token (PAT)](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token) with `repo` permissions. When prompted for your password during the `git push`, use the PAT instead.

Alternatively, you can configure Git to use a credential helper. However, for a simple push in Colab, using the PAT when prompted is often sufficient.

In [ ]:
from getpass import getpass
import os

# -----------------------------
# 1️⃣ GitHub info
GIT_USERNAME = "saymon800"
GIT_EMAIL = "kaaziisaymon58@gmail.com"
REPO_URL = "https://github.com/kazisaymon/Pdf-assistant.git"
PROJECT_PATH = "/content/ai-pdf-assistant"  # Project folder path

# GitHub Personal Access Token
GIT_TOKEN = getpass("Enter your GitHub Personal Access Token: ")

# -----------------------------
# 2️⃣ Create Project Folder
os.makedirs(PROJECT_PATH, exist_ok=True)
print("Project folder created at:", PROJECT_PATH)

# -----------------------------
# 3️⃣ Create Files
# app.py
app_code = """
import streamlit as st

st.title("📄 AI PDF Assistant")
st.write("Modern AI PDF Assistant App placeholder")
"""
with open(os.path.join(PROJECT_PATH, "app.py"), "w") as f:
    f.write(app_code)

# requirements.txt
requirements = """
streamlit
pdfplumber
sentence-transformers
faiss-cpu
transformers
torch
numpy
pandas
scikit-learn
tqdm
pyngrok
reportlab
"""
with open(os.path.join(PROJECT_PATH, "requirements.txt"), "w") as f:
    f.write(requirements.strip())

# README.md
readme = """
# AI PDF Assistant

A modern Streamlit app to:

- Upload PDF
- Generate summaries
- Ask questions (Q&A)
- Download summary as TXT or PDF
"""
with open(os.path.join(PROJECT_PATH, "README.md"), "w") as f:
    f.write(readme.strip())

# .gitignore
gitignore = """
__pycache__/
*.pyc
*.pkl
*.ipynb_checkpoints/
.env
"""
with open(os.path.join(PROJECT_PATH, ".gitignore"), "w") as f:
    f.write(gitignore.strip())

print("Files created successfully!")

# -----------------------------
# 4️⃣ Git Init, Config, Commit, Push
os.chdir(PROJECT_PATH)

# Git init
os.system("git init")

# Git config
os.system(f"git config user.name '{GIT_USERNAME}'")
os.system(f"git config user.email '{GIT_EMAIL}'")

# Add and commit
os.system("git add .")
os.system("git commit -m 'Initial commit: Modern AI PDF Assistant'")

# Set remote with token
remote_url_with_token = REPO_URL.replace("https://", f"https://{GIT_USERNAME}:{GIT_TOKEN}@")
os.system(f"git remote add origin {remote_url_with_token}")

# Rename branch to main
os.system("git branch -M main")

# Push to GitHub
os.system("git push -u origin main")

print("✅ All done! Check your GitHub repo.")
